<a href="https://colab.research.google.com/github/vsingh9076/Computer_Vision/blob/main/PyTesseract/PDF_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting text/tables from scanned PDF files

In [ ]:
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import os
import fitz

In [ ]:
# Create a function to convert the PDF to images
def convert_to_images(input_file):
    images = convert_from_path(input_file)
    for image in images:
      output_file = f"page_{images.index(image)}.png"
      image.save(output_file, 'PNG')

# Create a function to read text from images
def image_to_text(image_path):
    # Read the image
    img = Image.open(image_path)
    # Extract the text from the image
    text = pytesseract.image_to_string(img)
    print(text)
    return text

In [ ]:
# Find the PDF path
pdf_path = '/content/SAMPLETEXT.pdf'

In [ ]:
convert_to_images(pdf_path)
#image_to_text("/content/PDF_image.png")

In [ ]:
import pandas as pd
def extract_data_from_table_ocr(image_path, output_csv_path):
    from img2table.ocr import TesseractOCR
    from img2table.document import Image

    # Instantiation of OCR
    ocr = TesseractOCR(n_threads=1, lang="eng")

    # Instantiation of image document
    doc = Image(image_path)

    # Table extraction
    extracted_tables_all = doc.extract_tables(ocr=ocr,
                                        implicit_rows=False,
                                        borderless_tables=False,
                                        min_confidence=50)
    print("length : " , len(extracted_tables_all))
    if len(extracted_tables_all)>0:
      extracted_tables = extracted_tables_all[0]
      frames = []
      for table in extracted_tables_all:
          try:
              frames.append(table.df)
          except:
              frames.append(table)

      result = pd.concat(frames)

      result.to_csv(output_csv_path, index=False, header = True)

def get_all_tables_into_csv(image_folder, output_folder):
  # Iterate through image files in the folder
  for filename in os.listdir(image_folder):
      if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
          image_path = os.path.join(image_folder, filename)

          # Generate a corresponding CSV file path
          base_filename, _ = os.path.splitext(filename)
          output_csv_path = os.path.join(output_folder, f"{base_filename}.csv")

          # Process and save data to the CSV file
          extract_data_from_table_ocr(image_path, output_csv_path)
          print(f"Processed: {filename} -> {output_csv_path}")
  return None

In [ ]:
get_all_tables_into_csv("/content/","/content/")

length :  3
Processed: page_3.png -> /content/page_3.csv
length :  2
Processed: page_5.png -> /content/page_5.csv
length :  2
Processed: page_2.png -> /content/page_2.csv
length :  0
Processed: page_1.png -> /content/page_1.csv
length :  2
Processed: page_4.png -> /content/page_4.csv
length :  1
Processed: PDF_image.png -> /content/PDF_image.csv
length :  1
Processed: page_0.png -> /content/page_0.csv
